Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/ANNDL Challenge2

# Summary

In this notebook is shown how we built a model based on either the original series or the stationary series or the detrended series (it can be decided using a variable).

In this notebook is possible to see how we handled stationary / detrended series to allow the model to train. Furthermore, is possible to see how we handle the stationary / detrended predictions done by the model with the objective of obtaining back the prediction in the original domain.

Import libraries

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)


import matplotlib.pyplot as plt


In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

Choose the type of time series you want to use

In [ ]:
USING_ORIGINAL_SERIES = 1
USING_DETRENDED_SERIES = 2                                                      # If we decide to use the detrended series we don't consider and compute the seasonalities of the series
USING_STATIONARY_SERIES = 3

timeSeriesToUse = USING_DETRENDED_SERIES

if timeSeriesToUse not in [USING_ORIGINAL_SERIES, USING_DETRENDED_SERIES, USING_STATIONARY_SERIES]:
  raise Exception("Invalid value!")

Resources Paths

In [ ]:
trainDataFile = "./fullTrainData.npy"
validationDataFile = "./fullValidationData.npy"

Variables initialization

In [ ]:
trainData = np.load(trainDataFile, allow_pickle=True)
validationData = np.load(validationDataFile, allow_pickle=True)

# For each set we consider the original series, the rolling mean series, the detrended series, the periodicity, the stationary series
# because they are all necessary information to do the reverse process from stationary series to original series
labels = {'originalSeries': 0, 'rollingMeanSeries': 1, 'detrendedSeries': 2, 'seasonalityIndexOfSeries': 3, 'stationarySeries': 4}

### Manipulating the input dataset

In [ ]:
def build_sequences(series, window=300, stride=50, telescope=9):
  '''
  Split the single 'series' in multiple blocks of length 'window'. Each block is composed of a x part of length 'window' - 'telescope' and a y part of length 'telescope'.
  'Data
  'Window' is the length of the input of our network
  'Stride' is the number of samples to skip before starting the next window
  'Telescope' is the length of the output of our network
  '''
  blocks = []
  labels = []
  idx = 0

  # We divide a time series in multiple blocks
  # If the series length is not a multiple of the window size, then the remaining slice of the series is skipped
  while(idx + window <= len(series)):
    blocks.append(series[idx : (idx + window - telescope)])
    labels.append(series[(idx + window - telescope) : (idx + window)])
    idx += stride

  blocks = np.array(blocks)
  labels = np.array(labels)
  return np.array(blocks), np.array(labels)

In [ ]:
def compute_sequences_for_dataset(data, window, stride, telescope, minimum_length):
  '''
  Using the build_sequences function to increase the number of samples.
  'Data' is a list containing the starting series.
  'Window' is the length of the input of our network plus the telescope
  'Stride' is the number of samples to skip before starting the next window
  'Telescope' is the length of the output of our network
  'Minimum_length' is the minimum length that a series must have to be considered
  '''

  x = []
  y = []

  # Compute the various sequences for each stationary series in data
  for series in data:

    # We skip the series with length less than the chosen 'minimum_length'
    if (len(series) >= minimum_length):

      # If the series length is less than the window size we need to pad the series
      if (len(series) < window):
        padding_len = window - len(series)

        # We isolate the Y portion (the telescope part) to let us adding the padding at the end of the X portion
        temp = series[ len(series) - telescope : len(series)]

        # We isolate the X portion
        series  = series[0 : len(series) - telescope]

        # We create the padding as a series full of 2-s
        padding = np.full((padding_len), 2, dtype= 'float32')

        # Our resulting series is composed of the X portion, the padding and the Y portion
        series = np.concatenate((series, padding, temp))

      # the X portions of the blocks and the Y portions of the blocks are computed
      x_new, y_new = build_sequences(series, window, stride, telescope)

      for elem in x_new:
        x.append(elem)
      for elem in y_new:
        y.append(elem)

  x = np.array(x)
  y = np.array(y)

  return x, y

In [ ]:
# Define common variables

window = 218
telescope = 18

Using the build_sequences function to increase the number of samples in the training set

In [ ]:
# Using the build_sequences function to increase the number of samples in the training set

train_x = []
train_y = []
stride = 10

if timeSeriesToUse == USING_STATIONARY_SERIES:
  # Compute the various sequences for each stationary series in the training set
  data_to_use = trainData[labels['stationarySeries']]
elif timeSeriesToUse == USING_DETRENDED_SERIES:
  # Compute the various sequences for each detrended series in the training set
  data_to_use = trainData[labels['detrendedSeries']]
else:
  # Compute the various sequences for each original series in the training set
  data_to_use = trainData[labels['originalSeries']]

train_x, train_y = compute_sequences_for_dataset(data_to_use, window, stride, telescope, 80)

train_x = np.expand_dims(train_x, axis= 2)

print(train_x.shape)
print(train_y.shape)
print(f"Original number of sequences: {len(data_to_use)}")
print(f"Number of total sequences: {len(train_x)}")
print(f'By choosing a window equal to {window} and stride equal to {stride}, there are {len(train_x) - len(data_to_use)} more time series')

Using the build_sequences function to increase the number of samples in the validation set

In [ ]:
# Using the build_sequences function to increase the number of samples in the validation set

val_x = []
val_y = []

if timeSeriesToUse == USING_STATIONARY_SERIES:
  # Compute the various sequences for each stationary series in the validation set
  data_to_use = validationData[labels['stationarySeries']]

elif timeSeriesToUse == USING_DETRENDED_SERIES:
  # Compute the various sequences for each detrended series in the validation set
  data_to_use = validationData[labels['detrendedSeries']]

else:
  # Compute the various sequences for each original series in the validation set
  data_to_use = validationData[labels['originalSeries']]

# In this case we use a stride equal to window because we don't want the validation samples to be overlapped
val_x, val_y = compute_sequences_for_dataset(data_to_use, window, window, telescope, 50)

val_x = np.expand_dims(val_x, axis= 2)

print(val_x.shape)
print(val_y.shape)
print(f"Original number of sequences: {len(data_to_use)}")
print(f"Number of total sequences: {len(val_x)}")
print(f'By choosing a window equal to {window} and a stride equal to {window}, there are {len(val_x) - len(data_to_use)} more time series')

### Developing the network

In [ ]:
# Define common variables
input_shape = [window - telescope, 1]
output_shape = [telescope]
batch_size = 64
epochs = 200

save_the_model_on_file = False                                                  # Flag that says if it should save the model on file
model_file = "./tmp/model"                                                      # The model file path

Define the network

In [ ]:
def build_LSTM_model(input_shape, output_shape):

    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape)

    # Masking Layer
    x = tfkl.Masking(mask_value= 2, input_shape= input_shape)(input_layer)

    # Add two Bidirectional LSTM layer with 64 units
    x = tfkl.Bidirectional(tfkl.LSTM(256, return_sequences=True, name='lstm'), name='bidirectional_lstm')(x)
    x = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True, name='lstm'), name='bidirectional_lstm_2')(x)
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=False, name='lstm'), name='bidirectional_lstm_3')(x)

    output_layer = tfkl.Dense(output_shape[0])(x)

    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='LSTM_model')

    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

Train the network

In [ ]:
# Create the model
model = build_LSTM_model(input_shape, output_shape)
model.summary()

# Train the model
history = model.fit(
    x = train_x,
    y = train_y,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(val_x, val_y),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=3, factor=0.9, min_lr=1e-5)
    ]
).history

#Plot loss
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()


In [ ]:
if save_the_model_on_file:
  model.save(model_file)
  del model

  model = tfk.models.load_model(model_file)

### Inferences on predictions

In [ ]:
def splitToObtainFixedLenghtSeries(dataset, seasonalities, window, telescope):
  '''
  It permits to have blocks of length 'window', each one containing a fixed number of completed periods of the starting stationary series.
  Split each series included in 'dataset' in multiple blocks of fixed length equal to 'window'.
  Each block is composed of a X part and a Y part. Each block contains a discrete number of periods of the original series.
  'Dataset' is the list of input series.
  'Seasonalities' is a list containing the periodicity (a number) for each series
  'Window' is the length of each block (composed of two parts: X and Y)
  'Telescope' is the length of the Y portion of the block (The portion composed of the ""future"" samples to predict)
  '''

  x = []
  y = []
  resulting_seasonalities = []

  for index in range(0, len(dataset)):
    # If the periodicity of the series is greater than the window length it is skipped because no period can be entirely contained in a single block
    if seasonalities[index] > window:
      continue

    # The actual window size must be a multiple of the periodicity of the series
    multiple_window = int(window / seasonalities[index]) * seasonalities[index]
    # The remaining space in the block is filled using padding
    padding_length = window - multiple_window

    # Number of blocks resulting from this split for a single series
    num_blocks = int(len(dataset[index]) / multiple_window)

    # We avoid using the remaining slice (the last not full block)
    for block_ind in range(0, num_blocks):

      # Isolate the current block
      block = dataset[index][block_ind * multiple_window : (block_ind + 1) * multiple_window]

      # Isolate the real X portion of the block
      tmp = np.copy(block[ : multiple_window - telescope])

      # Create the pad sequence, full of 2-s
      pad = np.full(shape = padding_length, fill_value = 2)

      # Create the X portion of the block. It is composed of the real X portion of the block and the pad sequence
      x.append(np.append(tmp,pad))

      # Isolate the Y portion of the block
      y.append(np.copy(block[multiple_window - telescope : ]))

      # For each block we store the its seasonality (it is equal to the periodicity of the original series)
      resulting_seasonalities.append(seasonalities[index])

  return np.array(x), np.array(y), np.array(resulting_seasonalities)

In [ ]:
def reverseStationarity(predictions, original_series_val_x, detrended_series, seasonalities):
	'''
    Transform the stationary predictions in the predictions based on the original domain.
    'Predictions' is the list of the stationary predictions computed by the network.
    'Original_series_val_x' is the list of the X portion of the series based on the original domain.
	'Detrended_series' is the list of the corrispondent detrended X portions.
	'Seasonalities' is the list composed of the periodicity (it is a number) of each series.
    '''

	original_predictions = []													# The predicions done by the network moved in the original domain.
	rolling_window = 5

	for index in range(0, len(predictions)):

			detrended_prediction = []											# The computed detrended prediction (starting from the stationary prediction)
			original_prediction = []											# The computed original prediction (starting from the stationary prediction)
			tmp_detrended_series = np.copy(detrended_series[index])
			tmp_original_series_val_x = np.copy(original_series_val_x[index])

			# Compute the detrended prediction
			for prediction_index in range(0, len(predictions[index])):
					detrended_prediction.append(predictions[index][prediction_index] + tmp_detrended_series[len(tmp_detrended_series) - seasonalities[index]])
					tmp_detrended_series = np.concatenate((tmp_detrended_series, np.array([detrended_prediction[prediction_index]])))

			# Compute the rolling mean series and the original prediction
			for prediction_index in range(0, len(detrended_prediction)):
					rolling_mean = np.mean(np.array(tmp_original_series_val_x[len(tmp_original_series_val_x) - rolling_window : ]))
					original_prediction.append(predictions[index][prediction_index] + rolling_mean)
					tmp_original_series_val_x = np.concatenate((tmp_original_series_val_x, np.array([original_prediction[prediction_index]])))

			original_predictions.append(original_prediction)

	return np.array(original_predictions)

In [ ]:
def reverseDetrend(predictions, original_series_val_x):
    '''
    Transform the detrended predictions in the predictions based on the original domain.
    'Predictions' is the list of the detrended predictions computed by the network.
    'Original_series_val_x' is the list of the X portion of the series based on the original domain.
    '''

    original_predictions = []
    rolling_window = 5

    # For each detrended prediction we compute the corrispondent prediction in the original domain
    for index in range(0, len(predictions)):
        original_prediction = []
        tmp_original_series_val_x = np.copy(original_series_val_x[index])

        for prediction_index in range(0, len(predictions[index])):
            rolling_mean = np.mean(np.array(tmp_original_series_val_x[len(tmp_original_series_val_x) - rolling_window : ]))
            original_prediction.append(predictions[index][prediction_index] + rolling_mean)
            tmp_original_series_val_x = np.concatenate((tmp_original_series_val_x, np.array([original_prediction[prediction_index]])))

        original_predictions.append(original_prediction)

    return np.array(original_predictions)

Preparing the stationary series to use as input and the corrispondent Y portion composed of the future sample to predict

In [ ]:
stationarity_series_val_x = []                                                  # List containing the X portion of each computed block for the stationary series
stationarity_series_val_y = []                                                  # List containing the Y portion of each computed block for the stationary series
original_series_val_x = []                                                      # List containing the X portion of each computed block for the original series
original_series_val_y = []                                                      # List containing the Y portion of each computed block for the original series
detrend_series_val_x = []                                                       # List containing the X portion of each computed block for the detrended series
detrend_series_val_y = []                                                       # List containing the Y portion of each computed block for the detrended series

all_series_seasonalities = []                                                   # List containing the periodicity of each computed block (it is equivalent for stationary series, detrended series, original series and rolling mean series)

if timeSeriesToUse == USING_STATIONARY_SERIES:
  # Compute the various blocks of the stationary series and the corrispondent detrended series and original series and compute the periodicity of each block
  stationarity_series_val_x, stationarity_series_val_y, all_series_seasonalities = splitToObtainFixedLenghtSeries(validationData[labels['stationarySeries']], validationData[labels['seasonalityIndexOfSeries']], window, telescope)
  original_series_val_x, original_series_val_y, all_series_seasonalities = splitToObtainFixedLenghtSeries(validationData[labels['originalSeries']], validationData[labels['seasonalityIndexOfSeries']], window, telescope)
  detrend_series_val_x, detrend_series_val_y, all_series_seasonalities = splitToObtainFixedLenghtSeries(validationData[labels['detrendedSeries']], validationData[labels['seasonalityIndexOfSeries']], window, telescope)

  stationarity_series_val_x = np.expand_dims(stationarity_series_val_x, axis= 2)

elif timeSeriesToUse == USING_DETRENDED_SERIES:
  # Compute the sequences for each original series in the validation set, because they will be used to reverse the detrended predictions
  original_series_val_x, original_series_val_y = compute_sequences_for_dataset(validationData[labels['originalSeries']], window, window, telescope, 50)

else:
  original_series_val_x, original_series_val_y = val_x, val_y

Make the predictions using stationary series as input

In [ ]:
if timeSeriesToUse == USING_STATIONARY_SERIES:
    predictions = model.predict(stationarity_series_val_x)
else:
    predictions = model.predict(val_x)

Inference the predictions and compute the MSE and MAE

In [ ]:
# Remove the padding values from the X portion of the various blocks

if timeSeriesToUse == USING_STATIONARY_SERIES:
  iter = np.copy(original_series_val_x)
  original_series_val_x = []
  for series in iter:
    indices_padding = np.where(series == 2)
    original_series_val_x.append(np.delete(series, indices_padding))

  iter = np.copy(detrend_series_val_x)
  detrend_series_val_x = []
  for series in iter:
    indices_padding = np.where(series == 2)
    detrend_series_val_x.append(np.delete(series, indices_padding))

  # Transform the stationary prediction into prediction based in the original domain
  original_predictions = reverseStationarity(predictions, original_series_val_x, detrend_series_val_x, all_series_seasonalities)

elif timeSeriesToUse == USING_DETRENDED_SERIES:
  iter = np.copy(original_series_val_x)
  original_series_val_x = []
  for series in iter:
    indices_padding = np.where(series == 2)
    original_series_val_x.append(np.delete(series, indices_padding))

  # Transform the detrended prediction into prediction based in the original domain
  original_predictions = reverseDetrend(predictions, original_series_val_x)

else:
  original_predictions = predictions

# Calculate and print Mean Squared Error (MSE)
mean_squared_error = tfk.metrics.mean_squared_error(original_series_val_y.flatten(), original_predictions.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")

# Calculate and print Mean Absolute Error (MAE)
mean_absolute_error = tfk.metrics.mean_absolute_error(original_series_val_y.flatten(), original_predictions.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")

In [ ]:
def inspect_timeseries_predictions(X, Y, preds, num, telescope):
    '''
    Randomly plot a number 'num' of series composed by the known x portion and the true y portion and the predicted portion.
    'X' is a list containing all the available x portions.
    'Y' is a list containing all the available y portions (the true).
    'Preds' is a list containing all the predicted y portions (the predictions).
    'Num' is the number of series to plot.
    'Telescope' is the length of each y portion (it is valid also for the length of each predicted portion).
    '''

    figs, axs = plt.subplots(num, 1, sharex=True, figsize=(17,17))
    for i in range(0, num):
        idx=np.random.randint(0,len(X))
        axs[i].plot(np.arange(len(X[idx])), X[idx])
        axs[i].plot(np.arange(len(X[idx]), len(X[idx])+telescope), Y[idx], color='orange')
        axs[i].plot(np.arange(len(X[idx]), len(X[idx])+telescope), preds[idx], color='green')
        axs[i].set_ylim(-1,1)
    plt.show()

In [ ]:
# Plot some predictions
inspect_timeseries_predictions(original_series_val_x, original_series_val_y, original_predictions, 10, telescope)